## Here we train an GCN on the Various collected LCE's

Can also make reconstruction algo from base model, remove linear term -> Worse fit (2.06 vs 2.04) but could work

https://www.frontiersin.org/journals/artificial-intelligence/articles/10.3389/frai.2022.832909/full

In [ ]:
import straxen
import os, sys
import h5py
import tensorflow


tensorflow.config.optimizer.set_jit(True)
tensorflow.debugging.set_log_device_placement(False) 
tensorflow.config.set_visible_devices([], 'GPU') # Because It keeps running out of memory and doesnt remove the data
tensorflow.keras.backend.set_floatx('float32')

sys.path.append(os.path.join(os.getcwd(), 'ActualThesisWork'))
%load_ext autoreload
%autoreload 2
from functions import *

In [ ]:
# MC Data
patterns, positions = load_MC_data()

# Basic variables
dead_pmts = np.array([ 28, 108, 121, 144, 156, 164, 177])
not_dead_pmts = np.array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 157, 158, 159, 160, 161, 162, 163, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252])
pmt_radius = straxen.tpc_pmt_radius# 3  * 2.54 / 2 
tpc_radius = 66.4
max_z = straxen.tpc_z
I0_init_from_ULCE = 0.1 # This is never act used

# ------------------------------------ LCE ------------------------------------------------------
def get_minimal_LCE_coordinate_model(n_pmts, pmt_positions, n_pmt_pos = 1):
    """
    Generate's PMT Position coordinate, the main one used for now
    """
    conv_pmt_coords = keras.models.Sequential([
        keras.layers.RepeatVector(n_pmts, input_shape=(2,)),
        GetRadius((n_pmts, n_pmt_pos), keras.backend.variable(pmt_positions)),
    ], name='xy_to_pmt_coords')
    return conv_pmt_coords

def get_RLCEModel(pmt_pos_top,n_pmts, n_groups, group_slices, I0_init, guess,**kwargs):
    """
    Generate Radial LCE Model for fitting
    kwargs -> Overflow from old variables
    """
    # Input layer
    input_pos = keras.layers.Input(shape=(2))
    # Get different coordinates
    rho = get_minimal_LCE_coordinate_model(n_pmts = n_pmts, pmt_positions = pmt_pos_top, n_pmt_pos = 1)(input_pos)
    # Apply radial lce 
    m = RadialLCELayer(name='radial_lce',guess= guess, n_groups=n_groups, group_slices=group_slices)(rho)
    
    # Holds I0 for each PMT
    m = I0Layer(n_pmts, init_val= tensorflow.cast(I0_init, tensorflow.float32), name='QE_Layer')(m)
    # Normalization Layer 
    m = NormalizationLayer(name='Normalization')(m)
    return keras.models.Model(inputs=input_pos, outputs=m, name="Radial_LCE")

npmt_pd = np.array([ 1.6263583e+00,  9.4909382e+00, -4.1542644e-05,  7.7986442e-03])
I0s = np.array([0.004235879983752966, 0.004732195753604174, 0.00466069346293807, 0.004735287744551897, 0.004843718837946653, 0.0036560967564582825, 0.004791035782545805, 0.004351622425019741, 0.004277762491255999, 0.003982590511441231, 0.0036782578099519014, 0.004139808006584644, 0.003963056020438671, 0.004514740779995918, 0.0048719532787799835, 0.003294900059700012, 0.004271252546459436, 0.004213831853121519, 0.003675800282508135, 0.003923185635358095, 0.004053255543112755, 0.003687959862872958, 0.003949134610593319, 0.0039024786092340946, 0.003996435087174177, 0.0045274775475263596, 0.00314155500382185, 0.004692835733294487, 0.0017648281063884497, 0.003712652949616313, 0.0041131614707410336, 0.0039019035175442696, 0.0038532845210283995, 0.003392500802874565, 0.003896859474480152, 0.003918000962585211, 0.004090684931725264, 0.004209689795970917, 0.004247184377163649, 0.0050539360381662846, 0.004631971474736929, 0.0039286985993385315, 0.003714651335030794, 0.003667386481538415, 0.004035579971969128, 0.003805769607424736, 0.003694107523187995, 0.0038736388087272644, 0.004259074572473764, 0.0036939873825758696, 0.004129675682634115, 0.004015707410871983, 0.0042891716584563255, 0.005394953768700361, 0.0049893721006810665, 0.004004128277301788, 0.0036364563275128603, 0.0037317327223718166, 0.0037540518678724766, 0.004226438235491514, 0.003232576884329319, 0.004007598850876093, 0.003992777783423662, 0.00392210902646184, 0.003881508018821478, 0.0037358906120061874, 0.0034426788333803415, 0.004593325313180685, 0.004476095549762249, 0.004509852733463049, 0.00398320984095335, 0.0038602910935878754, 0.0038099722005426884, 0.0035987216979265213, 0.004056497476994991, 0.003779412480071187, 0.003909052349627018, 0.003880080534145236, 0.0037605699617415667, 0.0040501500479876995, 0.003605334088206291, 0.0037680561654269695, 0.003598541719838977, 0.0036737960763275623, 0.004982828162610531, 0.004214885178953409, 0.00394063163548708, 0.00341225927695632, 0.0037580307107418776, 0.003971084486693144, 0.004098842386156321, 0.003986097406595945, 0.004088491201400757, 0.004092586226761341, 0.003926937002688646, 0.004298168700188398, 0.004080779384821653, 0.003439064836129546, 0.003919169306755066, 0.0038085675332695246, 0.004447304643690586, 0.004079096484929323, 0.004161607474088669, 0.0035417259205132723, 0.004211203660815954, 0.003966113086789846, 0.004028564319014549, 0.0038740618620067835, 0.001677889609709382, 0.004096835386008024, 0.004131924360990524, 0.004420299082994461, 0.0038429093547165394, 0.003977605141699314, 0.004036216530948877, 0.0040572527796030045, 0.0034212754108011723, 0.004304381553083658, 0.004800576251000166, 0.003856675000861287, 0.0037602523807436228, 0.0017064257990568876, 0.003873104928061366, 0.003873534733429551, 0.003949661739170551, 0.004201176576316357, 0.0039010290056467056, 0.003948729019612074, 0.004660475999116898, 0.003974431660026312, 0.004105430096387863, 0.003848433494567871, 0.0039720600470900536, 0.004035878926515579, 0.005145389586687088, 0.004992862232029438, 0.004138431046158075, 0.0038946077693253756, 0.0037270355969667435, 0.003934138920158148, 0.004055908881127834, 0.004122219048440456, 0.00413487758487463, 0.004022635985165834, 0.0016843285411596298, 0.0039187888614833355, 0.004069270566105843, 0.004088082350790501, 0.0038323558401316404, 0.003933216445147991, 0.0041273306123912334, 0.00391603447496891, 0.003779760794714093, 0.0038304287008941174, 0.003971415106207132, 0.003770100185647607, 0.0016588973812758923, 0.003752924269065261, 0.0038747305516153574, 0.004287980031222105, 0.004396375268697739, 0.004206789191812277, 0.004279491025954485, 0.0038022841326892376, 0.0016486833337694407, 0.003921540919691324, 0.00393262505531311, 0.0036610763054341078, 0.005085785873234272, 0.00415172940120101, 0.0035354148130863905, 0.004259951412677765, 0.0040181828662753105, 0.003962669055908918, 0.004295114893466234, 0.004105494357645512, 0.00389536889269948, 0.0016863005002960563, 0.004174431320279837, 0.0035444151144474745, 0.0036017769016325474, 0.0038141191471368074, 0.0036408151499927044, 0.004543586168438196, 0.004658541176468134, 0.0037312631029635668, 0.0039724307134747505, 0.0036708605475723743, 0.003768590744584799, 0.003977857064455748, 0.003958065528422594, 0.00391046516597271, 0.004013632424175739, 0.004107540939003229, 0.004054495599120855, 0.003917690832167864, 0.0037647797726094723, 0.003846473526209593, 0.005038938019424677, 0.005282974801957607, 0.003871278138831258, 0.004100256133824587, 0.003908885642886162, 0.004233073443174362, 0.003930484410375357, 0.0037340044509619474, 0.0039030059706419706, 0.0037919552996754646, 0.00414620153605938, 0.003158329986035824, 0.0034461934119462967, 0.004152875859290361, 0.005023165140300989, 0.005013687536120415, 0.004022886045277119, 0.0038592142518609762, 0.003895816160365939, 0.004128684755414724, 0.0037005161866545677, 0.00402730843052268, 0.003608384868130088, 0.0027551278471946716, 0.004201559349894524, 0.0035797676537185907, 0.003656105836853385, 0.004713757894933224, 0.003788005094975233, 0.004254915285855532, 0.0041169957257807255, 0.004034965764731169, 0.0036038379184901714, 0.003389847930520773, 0.003905089572072029, 0.0038801683112978935, 0.0035936341155320406, 0.004151667468249798, 0.0040817963890731335, 0.004347157198935747, 0.004485994577407837, 0.00449867220595479, 0.004051011521369219, 0.0041497559286653996, 0.00477506872266531, 0.004078592173755169, 0.004272844642400742, 0.0041829245164990425, 0.005090698599815369, 0.003888470819219947, 0.004583718255162239, 0.005087660159915686, 0.0047324057668447495, 0.0045408145524561405, 0.0037471496034413576])

radial_lce = get_RLCEModel(pmt_pos_top[:,2:],n_pmts=n_pmts, n_groups=1, group_slices=np.arange(n_pmts), 
                        I0_init=I0_init_from_ULCE, guess = npmt_pd.tolist())
radial_lce.layers[-2].set_weights([I0s])

# ------------------------------------ LCE trained on Gaussian ---------------------------------------
# Ie rLCE retrained with Pienaar's Gaussian weights
npmt_pd = 
I0s = 

radial_lce = get_RLCEModel(pmt_pos_top[:,2:],n_pmts=n_pmts, n_groups=1, group_slices=np.arange(n_pmts), 
                        I0_init=I0_init_from_ULCE, guess = npmt_pd.tolist())
radial_lce.layers[-2].set_weights([I0s])

# ------------------------------------ LCE full retrain w Gaussian ---------------------------------------
# Ie rLCE with Gaussian weights fully retrained
npmt_pd = 
I0s = 

radial_lce = get_RLCEModel(pmt_pos_top[:,2:],n_pmts=n_pmts, n_groups=1, group_slices=np.arange(n_pmts), 
                        I0_init=I0_init_from_ULCE, guess = npmt_pd.tolist())
radial_lce.layers[-2].set_weights([I0s])


# ------------------------------- Pienaar LCE -----------------------------------------------------
with tf.keras.utils.custom_object_scope({'_logl_loss': _logl_loss}):
    model = tf.keras.models.load_model("/Code/LCE_SR1/LCE_Neural_SR1")
model_pattern = tf.keras.Model(model.inputs[0],
                                model.get_layer('normalization_1').output)

In [ ]:
# Def GCN

learning_rate = 5e-3
# define the model
X_in = Input(shape = (253,)) 
X = GraphConstrained(adj = adj[0], activation='elu')(X_in)
X = GraphConstrained(adj = adj[1], activation='elu')(X)
X = GraphConstrained(adj = adj[2], activation='elu')(X)
X = GraphConstrained(adj = adj[3], activation='elu')(X)
X = GraphConstrained(adj = adj[4], activation='elu')(X)
X = GraphConstrained(adj = adj[5], activation='elu')(X)
X = Dense(20, activation = 'elu')(X)
X = Dense(2, activation='tanh')(X)
output = FG_Squircular_to_disk(67.)(X) # map to circle of 67 cm
GCN = Model(inputs=X_in, outputs=output)
optimizer = Adam(learning_rate=learning_rate)
GCN.compile(optimizer = optimizer,
              loss ='mean_squared_error')
GCN.summary()

### And train one on each LCE 

### Perf Metrics

- MC Deviation

Real Data

- Hotspot by Gaussian

- Perp Wire resolution